In [10]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [11]:
import mlflow

In [12]:
num_classes = 10
input_shape = (28, 28, 1)

In [13]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [14]:
# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [15]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [16]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

In [17]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

with mlflow.start_run(run_name = "mnist"):
    batch_size = 128
    epochs = 4
    
    mlflow.log_param("batch_size",batch_size)
    mlflow.log_param("epochs",epochs)
    mlflow.log_param("optimizer","adam")
    
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)
    score = model.evaluate(x_test, y_test, verbose=0)
    
    test_loss = score[0]
    test_accuracy = score[1]
    
    mlflow.log_metric("loss", test_loss)
    mlflow.log_metric("accuracy", test_accuracy)
    
    print("Test loss:", test_loss)
    print("Test accuracy:", test_accuracy)

Epoch 1/4
422/422 [==============================] - 16s 37ms/step - loss: 0.3711 - accuracy: 0.8874 - val_loss: 0.0849 - val_accuracy: 0.9770
Epoch 2/4
422/422 [==============================] - 16s 37ms/step - loss: 0.1116 - accuracy: 0.9657 - val_loss: 0.0570 - val_accuracy: 0.9838
Epoch 3/4
422/422 [==============================] - 16s 37ms/step - loss: 0.0871 - accuracy: 0.9737 - val_loss: 0.0466 - val_accuracy: 0.9862
Epoch 4/4
422/422 [==============================] - 16s 37ms/step - loss: 0.0707 - accuracy: 0.9784 - val_loss: 0.0432 - val_accuracy: 0.9883
Test loss: 0.038537461310625076
Test accuracy: 0.9883000254631042


score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])